<a href="https://colab.research.google.com/github/AdrianaUP/News-query_RPP-lab/blob/main/notebooks/task1_rpp_news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
print("Notebook inicializado para Task 1")


Notebook inicializado para Task 1


In [2]:
pip install feedparser


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.5/81.5 kB 3.1 MB/s eta 0:00:00
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=8777059165fc0a3b23f7a60f6d20ee8962a8a2ed36497bc25905113fb6578a3a
  Stored in directory: /root/.cache/pip/wheels/03/f5/1a/23761066dac1d0e8e683e5fdb27e12de53209d05a4a37e6246
Successfully built sgmllib3k


In [3]:
import feedparser
import pandas as pd

rss_url = "https://rpp.pe/rss"
feed = feedparser.parse(rss_url)

news_items = []
for entry in feed.entries[:50]:
    news_items.append({
        "title": entry.title,
        "description": entry.description,
        "link": entry.link,
        "published": entry.published
    })

df_news = pd.DataFrame(news_items)
df_news.head()


,title,description,link,published
0,Petroperú designa a José Manuel Rodríguez Haya...,"Antes de su designación, se desempeñaba como g...",https://rpp.pe/economia/economia/petroperu-des...,"Fri, 24 Oct 2025 19:52:03 -0500"
1,Cúal fue el último temblor en México hoy 24 de...,Cuál es el ultimo temblor en México y CDMX reg...,https://rpp.pe/mundo/mexico/cual-fue-el-ultimo...,"Wed, 22 Oct 2025 06:42:35 -0500"
2,Sensibilidad y reflexión: la profundidad que t...,Una vida reflexiva no es un simple pasatiempo ...,https://rpp.pe/columnistas/ricardoleninalfredo...,"Fri, 24 Oct 2025 19:41:20 -0500"
3,Al Nassr vs Al Hazm con Cristiano Ronaldo: ¿a ...,El equipo de Cristiano Ronaldo buscará su sext...,https://rpp.pe/futbol/futbol-mundial/al-nassr-...,"Fri, 24 Oct 2025 19:37:55 -0500"
4,Abogado de Guillermo Bermejo califica al congr...,El defensor legal Ronald Atencio cuestionó la ...,https://rpp.pe/politica/judiciales/abogado-de-...,"Fri, 24 Oct 2025 19:12:22 -0500"


In [4]:
pip install tiktoken


In [5]:
import tiktoken

# Usaremos el codificador de OpenAI compatible con modelos como GPT-3.5
encoding = tiktoken.get_encoding("cl100k_base")

# Tomamos el texto completo de la primera noticia (título + descripción)
sample_text = df_news.loc[0, "title"] + " " + df_news.loc[0, "description"]

# Tokenizamos
tokens = encoding.encode(sample_text)

# Mostramos cantidad de tokens
print(f"Número de tokens: {len(tokens)}")


Número de tokens: 46


In [6]:
def count_tokens(text):
    return len(encoding.encode(text))

df_news["full_text"] = df_news["title"] + " " + df_news["description"]
df_news["num_tokens"] = df_news["full_text"].apply(count_tokens)

df_news[["title", "num_tokens"]].head()


,title,num_tokens
0,Petroperú designa a José Manuel Rodríguez Haya...,46
1,Cúal fue el último temblor en México hoy 24 de...,74
2,Sensibilidad y reflexión: la profundidad que t...,131
3,Al Nassr vs Al Hazm con Cristiano Ronaldo: ¿a ...,55
4,Abogado de Guillermo Bermejo califica al congr...,75


In [7]:
pip install sentence-transformers


In [8]:
from sentence_transformers import SentenceTransformer

# Cargar el modelo
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
# Generar embeddings
embeddings = model.encode(df_news["full_text"].tolist(), show_progress_bar=True)

# Verificar forma del primer embedding
print(f"Dimensión del embedding: {embeddings[0].shape}")


Batches:   0%|          | 0/2 [00:00<?, ?it/s]

Dimensión del embedding: (384,)


In [10]:
import numpy as np

# Convertir a lista de listas para guardar en DataFrame
df_news["embedding"] = [emb.tolist() for emb in embeddings]
